# Prototype code for Coder Speciation class
The SimpleSolnModel class will be extended as outlined below:
- Basis and non-basis (dependent species) endmember properties are defined using StdStateMdel
- A SpeciationSolnModel class is initialized with $c$ components (basis species) and $s$ non-basis species
- A model expression for the Gibbs free energy of solution is added that depends only on $T$, $P$, and mole numbers of the basis components
  - This expression should be optimzed to make compositional derivatives as compact as possible. This will dramatically lesson code generation and compilation time.
- When the class is code printed it generates the speciation code at the cython level, using scipy.optimize.minimize as a solver.  Eventually, this method should be replaced by a pure C implementation so that the code can be cleanly exported. 
  - A flag has been added to the class to avoid generation of compositional derivative code that is not currently used by the Equilibrate class.  The flag is added to the create_code_module method of the SimpleSoln class as shown below.
  - Only the bold derivatives in the list below are required.  The rest are optional.
  - dn_g_list: **dgdn**, **d2gdndt**, **d2gdndp**, **d3gdndt2**, **d3gdndtdp**, **d3gdndp2**, d4gdndt3, d4gdndt2dp, d4gdndtdp2, d4gdndp3  
  - d2n_g_list: **d2gdn2**, **d3gdn2dt**, **d3gdn2dp**, d4gdn2dt2, d4gdn2dtdp, d4gdn2dp2, d5gdn2dt3, d5gdn2dt2dp, d5gdn2dtdp2, d5gdn2dp3  
  - d3n_g_list: **d3gdn3**, d4gdn3dt, d4gdn3dp, d5gdn3dt2, d5gdn3dtdp, d5gdn3dp2, d6gdn3dt3, d6gdn3dt2dp, d6gdn3dtdp2, d6gdn3dp3
- Question that needs to be addressed is how does this algorithm scale?  
 - Will generation and compilation be too slow for 10 basis components and say 100 non-basis species?
 - Will the scipy.optimize.minimize method converge as quickly or as often as the number of basis species increases?
 - Should we insert tests for speciation convergence and abort the computation if these fail?

In [ ]:
import numpy as np
from os import path
import scipy.optimize as opt
import sys
import fileinput
import sympy as sym
from thermoengine import coder, core, phases, model, equilibrate
sym.init_printing()

In [ ]:
t = 1300.0  # K
p =    1.0  # bars
module_type = 'calib' # 'fast'

# Gas Endmembers (coder)

In [ ]:
modelCD = coder.StdStateModel()

In [ ]:
GTP = sym.symbols('GTP')
params = [('GTP','J',GTP)]
modelCD.add_expression_to_model(GTP, params)

In [ ]:
modelCD.set_module_name('gas_species')

In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

### Basis

In [ ]:
param_dict = {'Phase':'H','Formula':'H(1)','T_r':298.15,'P_r':1.0,'GTP':-100000.0}
result = modelCD.create_code_module(phase=param_dict.pop('Phase', None),
                                    formula=param_dict.pop('Formula', None),
                                    params=param_dict,
                                    module_type=module_type,
                                    silent=True)
%cp gas_species.pyx endmembers.pyx
file_name = '"H_gas_species_calib.c"' if module_type == 'calib' else '"H_gas_species_calc.c"'

In [ ]:
param_dict = {'Phase':'Al','Formula':'Al(1)','T_r':298.15,'P_r':1.0,'GTP':-200000.0}
result = modelCD.create_code_module(phase=param_dict.pop('Phase', None),
                                    formula=param_dict.pop('Formula', None),
                                    params=param_dict,
                                    module_type=module_type,
                                    silent=True)
%cat gas_species.pyx >> endmembers.pyx
file_name += ', "Al_gas_species_calib.c"' if module_type == 'calib' else ', "Al_gas_species_calc.c"'

### Non-basis

In [ ]:
param_dict = {'Phase':'AlH','Formula':'H(1)Al(1)','T_r':298.15,'P_r':1.0,'GTP':-160000.0}
result = modelCD.create_code_module(phase=param_dict.pop('Phase', None),
                                    formula=param_dict.pop('Formula', None),
                                    params=param_dict,
                                    module_type=module_type,
                                    silent=True)
%cat gas_species.pyx >> endmembers.pyx
file_name += ', "AlH_gas_species_calib.c"' if module_type == 'calib' else ', "AlH_gas_species_calc.c"'

In [ ]:
param_dict = {'Phase':'Al2H','Formula':'H(1)Al(2)','T_r':298.15,'P_r':1.0,'GTP':-400000.0}
result = modelCD.create_code_module(phase=param_dict.pop('Phase', None),
                                    formula=param_dict.pop('Formula', None),
                                    params=param_dict,
                                    module_type=module_type,
                                    silent=True)
%cat gas_species.pyx >> endmembers.pyx
file_name += ', "Al2H_gas_species_calib.c"' if module_type == 'calib' else ', "Al2H_gas_species_calc.c"'

In [ ]:
param_dict = {'Phase':'AlH2','Formula':'H(2)Al(1)','T_r':298.15,'P_r':1.0,'GTP':-400000.0}
result = modelCD.create_code_module(phase=param_dict.pop('Phase', None),
                                    formula=param_dict.pop('Formula', None),
                                    params=param_dict,
                                    module_type=module_type,
                                    silent=True)
%cat gas_species.pyx >> endmembers.pyx
file_name += ', "AlH2_gas_species_calib.c"' if module_type == 'calib' else ', "AlH2_gas_species_calc.c"'

# Build the endmembers

In [ ]:
%cp endmembers.pyx gas_species.pyx
with open('gas_species.pyxbld', 'r') as f:
    fold = f.read()
    f.close()
if module_type == 'calib':
    fnew = fold.replace("'AlH2_gas_species_calib.c'", file_name)
else:
    fnew = fold.replace("'AlH2_gas_species_calc.c'", file_name)
with open('gas_species.pyxbld', 'w') as f:
    f.write(fnew)
    f.close()

In [ ]:
import gas_species
%cd ..

In [ ]:
if module_type == 'calib':
    modelDB = model.Database(database="CoderModule", calib=True, phase_tuple=('gas_species', {
        'H':['H','pure'],
        'Al':['Al','pure'],
        'AlH':['AlH','pure'],
        'Al2H':['Al2H','pure'],
        'AlH2':['AlH2', 'pure']
    }))
else:
    modelDB = model.Database(database="CoderModule", calib=False, phase_tuple=('gas_species', {
        'H':['H','pure'],
        'Al':['Al','pure'],
        'AlH':['AlH','pure'],
        'Al2H':['Al2H','pure'],
        'AlH2':['AlH2', 'pure']
    }))

In [ ]:
H = modelDB.get_phase('H')
Al = modelDB.get_phase('Al')
AlH = modelDB.get_phase('AlH')
Al2H = modelDB.get_phase('Al2H')
AlH2 = modelDB.get_phase('AlH2')
mu0 = np.array([
    H.gibbs_energy(t,p), 
    Al.gibbs_energy(t,p), 
    AlH.gibbs_energy(t,p), 
    Al2H.gibbs_energy(t,p), 
    AlH2.gibbs_energy(t,p)
])
mu0_b = mu0[0:2]
mu0_s = mu0[2:]
mu0_b, mu0_s

 # Conversion Matrices

In [ ]:
C = np.array([
    H.props['element_comp'][0],
    Al.props['element_comp'][0],
    AlH.props['element_comp'][0],
    Al2H.props['element_comp'][0],
    AlH2.props['element_comp'][0]
])
elm_sys_ind = np.where(np.sum(C,axis=0) > 0)[0]
elm_sys = [core.chem.PERIODIC_ORDER[i] for i in elm_sys_ind]
C = C[:,elm_sys_ind]
elm_sys, C

In [ ]:
Cb = C[0:2,:]
Cs = C[2:,:]
Cb, Cs

In [ ]:
R = np.matmul(Cs, Cb.T)
R

In [ ]:
Q = np.exp(-(mu0_s - np.matmul(R,mu0_b))/(8.3143*t))
Q

# Test Solution method for Eq #15

In [ ]:
def sys_eqns(n, e, Q, R, Cb, Cs, print_species=False):
    nT = np.sum(e)
    x = n/nT
    prod = []
    for i, y in enumerate(Q):
        yy = 1.0
        for j, xx in enumerate(x):
            if R[i,j] != 0.0:
                yy *= xx**R[i,j]
        prod.append(yy)
    prod = np.diag(prod)
    if print_species:
        print (nT*np.matmul(Q.T,prod))
    result = np.matmul(Cb.T,n) + nT*np.matmul(Cs.T,np.matmul(Q.T,prod)) - e
    return np.matmul(result.T, result)

In [ ]:
e = np.array([2,2])
sys_eqns(np.array([1,1]), e, Q, R, Cb, Cs)

In [ ]:
n0 = np.array([1,1])
result = opt.minimize(sys_eqns, n0, args=(e, Q, R, Cb, Cs), bounds=opt.Bounds(np.zeros(2), e, keep_feasible=True))
result

In [ ]:
print ('Basis species concentrations:')
print (result.x)
print ('Non-basis species concentrations:')
print ('Function value:', sys_eqns(np.array(result.x), e, Q, R, Cb, Cs, print_species=True))

# Construct Solution Phase (coder)

In [ ]:
c = len(elm_sys)
modelCD = coder.SimpleSolnModel(nc=c)

In [ ]:
n = modelCD.n
nT = modelCD.nT
X = n/nT

In [ ]:
T = modelCD.get_symbol_for_t()
mu = modelCD.mu

In [ ]:
mu_sp_AlH, mu_sp_Al2H, mu_sp_AlH2, R = sym.symbols('mu_sp_AlH mu_sp_Al2H mu_sp_AlH2 R')

In [ ]:
Q_AlH = sym.exp(-(mu_sp_AlH-mu[0]-mu[1])/R/T)
Q_Al2H = sym.exp(-(mu_sp_Al2H-mu[0]-2*mu[1])/R/T)
Q_AlH2 = sym.exp(-(mu_sp_AlH2-2*mu[0]-mu[1])/R/T)

In [ ]:
Q_AlH, Q_Al2H, Q_AlH2

In [ ]:
X_AlH  = Q_AlH*X[0]*X[1]
X_Al2H = Q_AlH*X[0]*X[0]*X[1]
X_AlH2 = Q_AlH*X[0]*X[1]*X[1]

In [ ]:
G_ss = (n.transpose()*mu)[0] + nT*(mu_sp_AlH*X_AlH + mu_sp_Al2H*X_Al2H + mu_sp_AlH2*X_AlH2)
G_ss

In [ ]:
S_config = sym.symbols('S_config')
S_config = 0
for i in range(0,c):
    S_config += X[i]*sym.log(X[i])
S_config += X_AlH*sym.log(X_AlH)
S_config += X_Al2H*sym.log(X_Al2H)
S_config += X_AlH2*sym.log(X_AlH2)
S_config *= -R*nT
S_config

In [ ]:
G = G_ss - T*S_config
G

# Alternative formulation for G Eq(22)
This alternative but equivalent formulation speeds up code generation and compilation by a factor of 10  
Code size is also reduced by a factor of three

In [ ]:
Galt = (n.transpose()*mu)[0]
Galt

In [ ]:
S_config = 0
for i in range(0,c):
    S_config += X[i]*sym.log(X[i])
S_config *= -R*nT
Galt += - T*S_config
Galt

In [ ]:
Galt += X_AlH*(mu[0]+R*T*sym.log(X[0])+mu[1]+R*T*sym.log(X[1]))
Galt += X_Al2H*(2*mu[0]+2*R*T*sym.log(X[0])+mu[1]+R*T*sym.log(X[1]))
Galt += X_AlH2*(mu[0]+R*T*sym.log(X[0])+2*mu[1]+2*R*T*sym.log(X[1]))
Galt

In [ ]:
modelCD.add_expression_to_model(Galt, [
    ('mu_sp_AlH', 'J', mu_sp_AlH),
    ('mu_sp_Al2H', 'J', mu_sp_Al2H),
    ('mu_sp_AlH2', 'J', mu_sp_AlH2) 
])

In [ ]:
modelCD.module = "gas_soln"

In [ ]:
formula = ''
convert = []
test = []
for ind,elm in enumerate(elm_sys):
    formula += elm + '[' + elm + ']'
    convert.append('['+str(ind)+']=['+elm+']')
    test.append('['+str(ind)+'] >= 0.0')
formula, convert, test

In [ ]:
modelCD.formula_string = formula
modelCD.conversion_string = convert
modelCD.test_string = test

In [ ]:
paramValues = {
    'mu_sp_AlH':mu0_s[0],
    'mu_sp_Al2H':mu0_s[1],
    'mu_sp_AlH2':mu0_s[2],
    'T_r':298.15,
    'P_r':1.0
}
endmembers = ['H_gas_species', 'Al_gas_species']
paramValues, endmembers

# Code the model

In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

In [ ]:
modelCD.create_code_module(phase="IdealGas", params=paramValues, endmembers=endmembers, 
                           prefix="cy", module_type=module_type, silent=False, minimal_deriv_set=True)

# Inject some Python into the Cython wrapper
This injection deals with the speciation calculation (Eq #15), which must be done prior to any solution property calculation

In [ ]:
sub_dict = { mu[0]:sym.symbols('mu_end[0]'), mu[1]:sym.symbols('mu_end[1]'), R:8.3143, T:'t'}
sub_dict = dict(sub_dict, **paramValues)
code_to_inject = \
'''
import math
import scipy.optimize as opt

def sys_eqns(n, e, Q, R, Cb, Cs):
    nT = n[0] + n[1]
    x = n/nT
    prod = []
    for i, y in enumerate(Q):
        yy = 1.0
        for j, xx in enumerate(x):
            if R[i,j] != 0.0:
                yy *= xx**R[i,j]
        prod.append(yy)
    prod = np.diag(prod)
    result = np.matmul(Cb.T,n) + nT*np.matmul(Cs.T,np.matmul(Q.T,prod)) - e
    return np.matmul(result.T, result)

class Storage:
    t = 0.0
    p = 0.0
    e = np.zeros(1)
    x = np.zeros(1)

def speciate(double t, double p, e):
    if Storage.t == t and Storage.p == p and np.array_equal(Storage.e, e):
        return Storage.x
    Storage.t = t
    Storage.p = p
    Storage.e = np.copy(e)
'''
code_to_inject += '    Cb = np.array(' + str(Cb.tolist()) + ')\n'
code_to_inject += '    Cs = np.array(' + str(Cs.tolist()) + ')\n'
code_to_inject += '    R = np.array(' + str(np.matmul(Cs, Cb.T).tolist()) + ')\n'
code_to_inject += '    Q = np.empty(' + str(Cs.shape[0]) + ')\n'
code_to_inject += '    mu_end = np.empty(' + str(Cb.shape[0]) + ')\n'
for i in range(0,Cb.shape[0]):
    code_to_inject += '    mu_end['+str(i)+'] = cy_IdealGas_gas_soln_calib_endmember_mu0('+str(i)+', t, p)\n'
for i,x in enumerate([Q_AlH, Q_Al2H, Q_AlH2]):
    code_to_inject += '    Q['+str(i)+'] = ' + sym.pycode(x.subs(sub_dict)) + '\n'
code_to_inject += \
'''
    n0 = e/2.0
    result = opt.minimize(sys_eqns, n0, args=(e, Q, R, Cb, Cs), bounds=opt.Bounds(np.zeros(2), e, keep_feasible=True))
    Storage.x = np.copy(result.x)
    return result.x

'''
code_to_add = '    np_array = speciate(t, p, np_array)'

In [ ]:
print (code_to_inject)
print (code_to_add)

In [ ]:
with open('gas_soln.pyx', 'r') as f:
    fold = f.read()
    f.close()
st = [
    'def cy_IdealGas_gas_soln_calib_g(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_dgdt(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_dgdp(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d2gdt2(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d2gdtdp(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d2gdp2(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d3gdt3(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d3gdt2dp(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d3gdtdp2(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d3gdp3(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_s(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_v(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_cv(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_cp(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_dcpdt(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_alpha(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_beta(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_K(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_Kp(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_dgdn(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d2gdndt(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d2gdndp(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d3gdndt2(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d3gdndtdp(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d3gdndp2(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d4gdndt3(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d4gdndt2dp(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d4gdndtdp2(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d4gdndp4(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d2gdn2(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d3gdn2dt(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d3gdn2dp(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d4gdn2dt2(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d4gdn2dtdp(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d4gdn2dp2(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d5gdn2dt3(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d5gdn2dt2dp(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d5gdn2dtdp2(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d5gdn2dp3(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d3gdn3(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d4gdn3dt(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d4gdn3dp(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d5gdn3dt2(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d5gdn3dtdp(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d5gdn3dp2(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d6gdn3dt3(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d6gdn3dt2dp(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d6gdn3dtdp2(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_calib_d6gdn3dp3(double t, double p, np_array):',
    'def cy_IdealGas_gas_soln_dparam_g(double t, double p, np_array, int index):',
    'def cy_IdealGas_gas_soln_dparam_dgdt(double t, double p, np_array, int index):',
    'def cy_IdealGas_gas_soln_dparam_dgdp(double t, double p, np_array, int index):',
    'def cy_IdealGas_gas_soln_dparam_d2gdt2(double t, double p, np_array, int index):',
    'def cy_IdealGas_gas_soln_dparam_d2gdtdp(double t, double p, np_array, int index):',
    'def cy_IdealGas_gas_soln_dparam_d2gdp2(double t, double p, np_array, int index):',
    'def cy_IdealGas_gas_soln_dparam_d3gdt3(double t, double p, np_array, int index):',
    'def cy_IdealGas_gas_soln_dparam_d3gdt2dp(double t, double p, np_array, int index):',
    'def cy_IdealGas_gas_soln_dparam_d3gdtdp2(double t, double p, np_array, int index):',
    'def cy_IdealGas_gas_soln_dparam_d3gdp3(double t, double p, np_array, int index):',
    'def cy_IdealGas_gas_soln_dparam_dgdn(double t, double p, np_array, int index):'
]
first = True
for x in st:
    if first:
        fnew = fold.replace(x, code_to_inject + x + '\n' + code_to_add)
        first = False
    else:
        fnew = fold.replace(x, x + '\n' + code_to_add)
    fold = fnew
with open('gas_soln.pyx', 'w') as f:
    f.write(fnew)
    f.close()

In [ ]:
import gas_soln
%cd ..

# Import the Solution Phase

In [ ]:
if module_type == 'calib':
    modelGas = model.Database(database="CoderModule", calib=True, 
                              phase_tuple=('gas_soln', {'Gas':['IdealGas','solution']}))
else:
    modelGas = model.Database(database="CoderModule", calib=False, 
                              phase_tuple=('gas_soln', {'Gas':['IdealGas','solution']}))
Gas = modelGas.get_phase('Gas')

In [ ]:
print (Gas.props['phase_name'])
print (Gas.props['formula'])
print (Gas.props['molwt'])
print (Gas.props['abbrev'])
print (Gas.props['endmember_num'])
print (Gas.props['endmember_name'])

In [ ]:
Gas.gibbs_energy(t,p,mol=np.array([2.,2.]))

# Perform Equilibrium Calculation

In [ ]:
phs_sys  = [Gas]
equil = equilibrate.Equilibrate(['H','Al'], phs_sys)

In [ ]:
state = equil.execute(t, p, bulk_comp=np.array([2,2]), debug=1)
state.print_state()